In [1]:
import pandas as pd
import urllib.request

In [2]:
preico_addr = '0x71c2683152b632fa1d515a3df067825e2b240216'
ico1_addr = '0xd48e165b0027e8055b3b7016a35078b23527a95d'
ico2_addr = '0x3aa5fa4fbf18d19548680a5f2bba061b18fed26b'

url_template = 'http://api.etherscan.io/api?module=account&action=txlist&sort=asc&address=%s'


In [3]:
def get_url(url):
    response = urllib.request.urlopen(url)
    data = response.read()
    return  data.decode('utf-8').replace('{"status":"1","message":"OK","result":', '').replace('}]}', '}]')

In [4]:
preico_json = get_url(url_template % preico_addr)
ico1_json = get_url(url_template % ico1_addr)
ico2_1_json = get_url(url_template % ico2_addr)
ico2_2_json = get_url((url_template+'&startblock=4263256&') % ico2_addr)


In [5]:
preico = pd.read_json(preico_json)
ico1 = pd.read_json(ico1_json)
ico2_1 = pd.read_json(ico2_1_json)
ico2_2 = pd.read_json(ico2_2_json)

ico = pd.concat([ico1, ico2_1, ico2_2])

In [6]:
#to ether from whei
preico['value'] /= 10**18
ico['value'] /= 10**18

In [7]:
ico.tail(2)

,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,from,gas,gasPrice,gasUsed,hash,input,isError,nonce,timeStamp,to,transactionIndex,value
1987,0x78e0caa79d17a0ddceea978fcf01f0f12c8a3df56ec5...,4274182,14,,5879830,0x23c2d9d773cc62596883a284ec5fd798eda28478,300000,24000000000,213188,0x14fb35e048c8a2ae74ccafd974a3100df357c51d80ef...,0x,0,0,2017-09-14 19:55:20,0x3aa5fa4fbf18d19548680a5f2bba061b18fed26b,100,0.200000
1988,0xde861865efb5452d51bbfa6d77d4a41e978e888c04f4...,4274189,7,,2313854,0xeffbb2643bb868be38f1135b3ffa8600972be56f,300000,24000000000,125453,0xaf2929964acef440e069cf6034f9d201ef6fa3e0c0de...,0x,0,1,2017-09-14 19:58:07,0x3aa5fa4fbf18d19548680a5f2bba061b18fed26b,64,64.093326


In [8]:
#transactions count (preico, ico)
preico.shape[0], ico.shape[0]

(1316, 17223)

In [9]:
#valid transactions (preico, ico)
preico_valid = preico[preico['isError']==0]
ico_valid = ico[ico['isError']==0]

preico_valid.shape[0], ico_valid.shape[0]

(1007, 13720)

In [10]:
#valid transactions percent (preico, ico)

preico_valid.shape[0] / preico.shape[0] * 100, \
    ico_valid.shape[0]/ico.shape[0]* 100

(76.51975683890578, 79.66091853916275)

In [11]:
#preico raised (valid and error)
preico.groupby('isError').sum().value

isError
0    5063.229048
1     903.947446
Name: value, dtype: float64

In [12]:
#ico raised (valid and error)
ico.groupby('isError').sum().value

isError
0    75073.841742
1    16752.566799
Name: value, dtype: float64

In [13]:
#avg incoming per transactions (preico, ico)

(preico_valid.sum().value / preico_valid.shape[0]), \
    (ico_valid.sum().value / ico_valid.shape[0])

(5.0280328179441565, 5.4718543543714526)

In [14]:
#avg incoming per person (preico, ico)

(preico_valid.sum().value / preico_valid['from'].unique().shape[0]), \
    (ico_valid.sum().value / ico_valid['from'].unique().shape[0])


(6.1150109271373978, 6.6839246565149866)

In [15]:
#top persons on preico
preico_valid.groupby('from').sum().sort_values('value', ascending=False)['value'].head(10)

from
0x074d2523c78e59dd0abcf61dd88f2354c0e5b697    500.000000
0xf84305d4077ec7e5546adf6f6c89b11e9b694bbc    489.000000
0x5aaf1c31254a6e005fba7f5ab0ec79d7fc2b630e    142.000000
0xe32ea43ee8670a04cd679c3bdde672188a11511f    136.320000
0x1d0738b927dfcbfbd59a9f0944bbd1860d3b9248    130.100000
0x4adc8d6d1ace99ce85438a111844c648babf814d    100.714000
0x8dbd7b3258be2824df1b3f87430d508638408cc1     70.321302
0x971a7659b8f1a887cc96414be36ca3fc885778d7     55.000000
0x7fc84138f145ba8f7bead5f970b621eaaf308238     52.000000
0xe334c2f2a6749efd1747bb28544751df51917e3c     50.000000
Name: value, dtype: float64

In [17]:
#top persons on preico
ico_valid.groupby('from').sum().sort_values('value', ascending=False)['value'].head(10)

from
0x31ad75479d7757a43e7ccc5bd72a78aa295f8754    15125.0
0xaac3b69a5197dd6b1cbab3674f9412a67d4d5ec9     3000.0
0xae748eb0d061a1121847fe9462bc6ed3c21b6865     1010.0
0x76edd365c44a612c91dd633662b7756adf3c46c2     1000.0
0x0464b244c6ccdce93bd2a510fb94f1240b4ad479     1000.0
0xfc2fe4c408a0af4c07d9a45453a9cd85cc042fac      800.0
0xebb6d58cd96d2617b23b19b8a4ae45a64c4502a7      667.0
0xeca935f795af72529d61b02113b60c19c5f4fdd7      500.0
0x18a19f00c2124495984776d701155c7c8b7d6625      500.0
0xd7cb7484ab22b47d788a34cd8035b9f7b512d371      465.1
Name: value, dtype: float64

In [18]:
#top persons by transactions count on preico
preico_valid.groupby('from').count().sort_values('value', ascending=False)['value'].head(10)

from
0x318ce896a1520dea2f1748a97b2b631bce44f378    6
0x77086b7ff405af6175f88ccc81c67f0b7c050846    5
0x5031129a4a1c0eaa0b98f989f9f3087c9a236451    4
0xf8c41fce8460a392d22f441835e653d2f9fa8caa    4
0x50fa7fdded5d6524d548d9f719cebdc52e0253bb    4
0x0299920d70dac3c5299d45a62ebf7c66448f2f97    4
0x4a6f39d8c90b6ef31e3191bfecf6727f4cc52358    4
0x84c14317b8152ec0317ae420ad8a77ac904eb523    4
0x74bf6ed447ae4d994a15c618b51f16b77936c6c6    3
0xf30e833ef51ef7c478f1411aeb1b3b81f28a5a58    3
Name: value, dtype: int64

In [19]:
#top persons by transactions count on ico
ico_valid.groupby('from').count().sort_values('value', ascending=False)['value'].head(10)

from
0x7ed1e469fcb3ee19c0366d829e291451be638e59    45
0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98    15
0xe7ff568d7ce43384f1069ad66ef9feeae5d41f08    11
0xc6d655252082ce78c723f6862559a3a86023db52    10
0x873104a06f5e8eb115be8c94e6a19ca3e56623d0    10
0x5031129a4a1c0eaa0b98f989f9f3087c9a236451     9
0xca2c8d9ba6ca96b3e0cc51c101d34a164900e7de     9
0x93272b1e5b5d2d642f5edc79f9a0cd93a1b11449     9
0xcf900b9803fe98bff7cbc71a0853f3b8d6de6480     8
0x7a414191d51e085e6efe698ca95904b9707d9b0c     8
Name: value, dtype: int64

In [20]:
#transactions of the addr with the biggest transactions count
ico_valid[ico_valid['from'] == '0x7ed1e469fcb3ee19c0366d829e291451be638e59'].head(10)[['timeStamp', 'value']], \
ico_valid[ico_valid['from'] == '0x7ed1e469fcb3ee19c0366d829e291451be638e59'].tail(10)[['timeStamp', 'value']]

(               timeStamp     value
 1229 2017-08-29 13:18:51  3.335000
 2114 2017-08-29 13:50:01  0.680000
 2435 2017-08-29 13:56:18  2.983000
 2632 2017-08-29 14:10:51  0.064171
 3048 2017-08-29 14:40:42  4.000000
 2168 2017-08-30 03:10:24  0.508491
 2690 2017-08-30 06:12:01  0.108740
 2767 2017-08-30 06:34:56  0.850000
 2971 2017-08-30 07:42:08  0.089858
 3215 2017-08-30 10:40:03  0.910555,                timeStamp     value
 9964 2017-09-11 17:00:06  0.092022
 73   2017-09-11 22:04:45  2.010000
 123  2017-09-12 01:02:04  1.000000
 208  2017-09-12 04:23:50  0.851069
 340  2017-09-12 10:24:37  0.060000
 708  2017-09-12 21:36:57  0.065902
 968  2017-09-13 08:08:21  1.000000
 1092 2017-09-13 11:44:43  0.235000
 1457 2017-09-13 23:17:18  2.023045
 1695 2017-09-14 09:23:51  0.050000)